In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import glob
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from collections import Counter

## READ DATA

In [2]:
taxonomy_files = glob.glob("controls/*_taxonomy.csv") 
pathways_files = glob.glob("controls/*_pathways.csv") 
metadata = pd.read_csv('controls/metadata_healthy.csv', index_col = [0], low_memory=False)

In [28]:
#metadata = metadata[metadata.age_category == 'adult']

In [3]:
### change pathways to unstratify and calculate relative abundance
PATHWAYS_DF = pd.DataFrame()

for file in pathways_files:
    pathways = pd.read_csv(file, index_col = [0])
    unstratified_columns = list(filter(lambda col: '|' not in col and 'unclassified' not in col, pathways.columns))
    unstratified_pathways = pathways[unstratified_columns]
    PATHWAYS_DF = pd.concat([PATHWAYS_DF, unstratified_pathways])

PATHWAYS_DF = PATHWAYS_DF.fillna(0)
PATHWAYS_DF.to_csv('pathways.csv')

In [5]:
TAXONOMY_DF = pd.DataFrame()
for file in taxonomy_files:
    taxonomy_df = pd.read_csv(file, index_col = [0])
    TAXONOMY_DF = pd.concat([TAXONOMY_DF, taxonomy_df])

TAXONOMY_DF = TAXONOMY_DF.fillna(0)
TAXONOMY_DF.to_csv('taxonomy.csv')

In [73]:
# Get overlapping samples between taxonomy, pathways and metadata
metadata_ids = set(metadata['sample_id'])
taxonomy_ids = set(TAXONOMY_DF.index)
pathways_ids = set(PATHWAYS_DF.index)
IDS = [metadata_ids, taxonomy_ids, pathways_ids]
keep_samples = list(set.intersection(*IDS))

In [74]:
PATHWAYS_DF = PATHWAYS_DF[PATHWAYS_DF.index.isin(keep_samples)]
TAXONOMY_DF = TAXONOMY_DF[TAXONOMY_DF.index.isin(keep_samples)]
metadata = metadata[metadata['sample_id'].isin(keep_samples)]

offset = metadata[['sample_id', 'number_reads']]

In [75]:
PATHWAYS_DF.shape, TAXONOMY_DF.shape, metadata.shape

((13112, 628), (13112, 1638), (13112, 141))

## FILTER FEATURES

In [103]:
def filter_prevalence(df, treshold = 0.05):
    '''features as columns'''
    df_binary = df.copy()
    df_binary[df_binary>0]=1
    df_binary_sum = df_binary.sum(axis=0)
    
    keep_features = df_binary_sum[df_binary_sum > df.shape[0]*treshold].index
    filtered_df = df[keep_features]
    
    return filtered_df

def filter_on_abundance(df, abundance_treshold = 1e-5):
    '''features as columns'''
    df_relab = df.div(df.sum(axis=1), axis=0)
    df_relab_mean = df_relab.mean()

    keep_features = df_relab_mean[df_relab_mean > abundance_treshold].index
    filtered_df = df[keep_features]
    
    return filtered_df

In [104]:
FILTERED_TAXONOMY = filter_on_abundance(filter_prevalence(TAXONOMY_DF))
FILTERED_PATHWAYS = filter_on_abundance(filter_prevalence(PATHWAYS_DF))

In [105]:
FILTERED_PATHWAYS.shape, FILTERED_TAXONOMY.shape

((13112, 311), (13112, 253))

In [ ]:
FILTERED_PATHWAYS.to_csv('input_data/controls_pathways.csv')
FILTERED_TAXONOMY.to_csv('input_data/controls_taxonomy.csv')
metadata.to_csv('input_data/controls_metadata.csv')

## TAXA AND FUNCTION DISTRIBUTION IN POPULATION

In [ ]:
taxonomy_binary = FILTERED_TAXONOMY.copy()
taxonomy_binary[taxonomy_binary>0]=1

pathways_binary = FILTERED_PATHWAYS.copy()
pathways_binary[pathways_binary>0]=1

fig, axes = plt.subplots(1, 2, figsize = (15, 3))
sns.histplot(taxonomy_binary.sum().values, ax=axes[0])
sns.histplot(pathways_binary.sum().values, ax=axes[1])

plt.suptitle('Numer of subjects where bacteria or patwhays is present')
plt.xlabel('n subjects')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (15, 3))
sns.histplot(taxonomy_binary.sum(axis=1).values, ax=axes[0])
sns.histplot(pathways_binary.sum(axis=1).values, ax=axes[1])

plt.suptitle('Number of bacteria or pathways per subject')
plt.xlabel('subjects')